# Instructor: Structured Outputs from LLMs

Welcome to this tutorial on `instructor`! This notebook will guide you through the essentials of using this powerful library to get structured, predictable, and validated data from Large Language Models (LLMs).

## What is `instructor`?

`instructor` is a Python library that makes it easy to get structured data, like JSON, from LLMs. Instead of hoping for the best with prompt engineering, `instructor` lets you define your desired output format using Pydantic models. This makes your code more reliable and easier to maintain.

## Why use `instructor`?

Working directly with LLMs can be challenging:

* **Unpredictable Outputs:** LLMs can format their responses differently each time.
* **Error-Prone Parsing:** Manually parsing JSON from a string can be brittle.
* **Validation Headaches:** You have to write extra code to check if the LLM's output is what you expected.

`instructor` solves these problems by:

* **Defining a Schema:** You define the structure of the data you want using Pydantic models.
* **Automatic Validation:** It ensures the LLM's output conforms to your Pydantic model.
* **Automatic Retries:** If the output is invalid, `instructor` can automatically retry the request and tell the LLM what went wrong.

This tutorial will cover:

* **Setup:** Installing the necessary libraries and loading API keys.
* **Core Concepts:** Basic extraction, validation and retries, and streaming.
* **Advanced Usage:** Working with different LLMs and nested Pydantic models.
* **A Practical Example:** Putting it all together to extract information from a block of text.

## 1. Setup

First, let's install the necessary libraries.

In [2]:
#%pip install instructor openai python-dotenv pydantic -q

Next, create a file named `.env` in the same directory as this notebook. This file will store your API keys securely. Your `.env` file should look like this:

```
OPENAI_API_KEY="your_openai_api_key_here"
# ANTHROPIC_API_KEY="your_anthropic_api_key_here" # Optional
```

Now, let's load these environment variables into our notebook.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") # Optional

## 2. Core Concepts

Now that we have everything set up, let's dive into the core concepts of `instructor`.

### 2.1. Basic Extraction with Pydantic

The fundamental idea behind `instructor` is to use Pydantic models to define the structure of the data you want to extract. Let's start with a simple example: extracting user information from a sentence.

In [4]:
import instructor
from openai import OpenAI
from pydantic import BaseModel

# 1. Define your desired data structure
class UserDetail(BaseModel):
    name: str
    age: int

# 2. Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# 3. Make a request with the `response_model` parameter
user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Jason is 25 years old."},
    ],
)

print(user)
print(f"User's name: {user.name}")
print(f"User's age: {user.age}")

name='Jason' age=25
User's name: Jason
User's age: 25


### 2.2. Validation and Retries

One of the most powerful features of `instructor` is its ability to validate the LLM's output and automatically retry if the validation fails. We can add validators to our Pydantic models to enforce specific rules.

Let's create a model with a validator that ensures a user is an adult. We'll also use `max_retries` to give the LLM a chance to correct itself.

In [5]:
from pydantic import BaseModel, Field, field_validator

class User(BaseModel):
    name: str
    age: int = Field(..., description="The age of the user")

    @field_validator("age")
    def validate_age(cls, v):
        if v < 18:
            raise ValueError("User must be an adult (18 or older)")
        return v

try:
    user = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=User,
        messages=[
            {"role": "user", "content": "Extract John is 17 years old."}
        ],
        max_retries=2,
    )
    print(user)
except Exception as e:
    print(f"Failed after retries: {e}")

Failed after retries: 1 validation error for User
age
  Value error, User must be an adult (18 or older) [type=value_error, input_value=17, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


### 2.3. Streaming Responses

`instructor` also supports streaming, which is useful for getting partial results in real-time. This is great for applications where you want to show a responsive UI.

To enable streaming, we set `stream=True` and use `Partial` from `instructor`.

In [6]:
from instructor import Partial

class User(BaseModel):
    name: str
    age: int

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=Partial[User],
    messages=[
        {"role": "user", "content": "Extract Sarah is 30 years old."}
    ],
    stream=True,
)

for user_chunk in stream:
    print(user_chunk)

name=None age=None
name=None age=None
name='' age=None
name='Sarah' age=None
name='Sarah' age=None
name='Sarah' age=None
name='Sarah' age=None
name='Sarah' age=30
name='Sarah' age=30


### 2.4. Working with Different LLMs

`instructor` is not limited to OpenAI. You can use it with other providers like Anthropic, Google, and even open-source models through Ollama.

Here's an example of how you might use `instructor` with Anthropic's Claude 3.

In [ ]:
# Note: You'll need to have the 'anthropic' library installed: %pip install anthropic
# and your ANTHROPIC_API_KEY set in your .env file.

# from anthropic import Anthropic
# 
# client_anthropic = instructor.from_anthropic(Anthropic())
# 
# class UserDetail(BaseModel):
#     name: str
#     age: int
# 
# user = client_anthropic.messages.create(
#     model="claude-3-opus-20240229",
#     response_model=UserDetail,
#     max_tokens=1024,
#     messages=[
#         {"role": "user", "content": "Extract Michael is 40 years old."},
#     ],
# )
# 
# print(user)

### 2.5. Nested Pydantic Models

You can also extract more complex, nested data by defining nested Pydantic models.

In [7]:
from typing import List

class Address(BaseModel):
    street: str
    city: str

class UserWithAddress(BaseModel):
    name: str
    age: int
    addresses: List[Address]

user = client.chat.completions.create(
    model="gpt-4o",
    response_model=UserWithAddress,
    messages=[
        {"role": "user", "content": "Extract John Doe is 30 and lives at 123 Main St in New York, NY."},
    ],
)

print(user)

name='John Doe' age=30 addresses=[Address(street='123 Main St', city='New York, NY')]


## 3. Putting It All Together: A Practical Example

Let's use what we've learned to extract information from a product review. We'll extract the product name, the rating, and a summary of the pros and cons.

In [9]:
from typing import List
from pydantic import BaseModel, Field

class ProductReview(BaseModel):
    product_name: str = Field(..., description="The name of the product being reviewed.")
    rating: int = Field(..., description="The star rating from 1 to 5.")
    pros: List[str] = Field(..., description="A list of positive aspects of the product.")
    cons: List[str] = Field(..., description="A list of negative aspects of the product.")

review_text = """
I recently bought the new 'SuperWidget 3000' and I'm very impressed!
It's incredibly fast and the battery life is amazing. I'd give it 5 stars.
The only downside is that it's a bit heavier than I expected.
"""

review = client.chat.completions.create(
    model="gpt-4o",
    response_model=ProductReview,
    messages=[
        {"role": "user", "content": f"Extract the following product review: {review_text}"},
    ],
)

import json
print(json.dumps(review.model_dump(), indent=2))

{
  "product_name": "SuperWidget 3000",
  "rating": 5,
  "pros": [
    "Incredibly fast",
    "Amazing battery life"
  ],
  "cons": [
    "A bit heavier than expected"
  ]
}


## Conclusion

Congratulations! You've learned the fundamentals of using `instructor` to get structured data from LLMs. We've covered:

* **Basic Extraction:** Using Pydantic models to define your desired output.
* **Validation and Retries:** Enforcing rules on the output and giving the LLM a chance to self-correct.
* **Streaming:** Getting partial results in real-time.
* **Advanced Usage:** Working with different LLMs and nested data.

`instructor` is a powerful tool that can make your AI-powered applications more reliable and robust. For more information, check out the official documentation and GitHub repository.